In [1]:
!pip install requests
!pip install numpy
!pip install pandas
!pip install selenium
!pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import re
import numpy as np
import pandas as pd
import time
import random


Reference: [here](https://www.geeksforgeeks.org/google-maps-selenium-automation-using-python/)

In [39]:
malls = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_pusat_perbelanjaan_di_Jakarta")
mall_s = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/ul/li')
for mall in mall_s:
    malls.append(mall.text)
driver.quit()
malls = [mall.replace('Mall', 'Mal').replace('Mal', 'Mall') for mall in malls]

In [41]:
krls = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Kategori:Stasiun_kereta_api_di_Jakarta")
krl_s = driver.find_elements(By.XPATH, '//*[@id="mw-pages"]/div/div/div/ul/li/a')
for krl in krl_s:
    krls.append(krl.text)
driver.quit()

In [56]:
tjs = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_koridor_Transjakarta")
tj_s = driver.find_elements(By.CSS_SELECTOR, 'tr>td:nth-child(2)>a.mw-redirect[href^="/wiki/"]')
for tj in tj_s:
    tjs.append('Halte Busway ' + tj.text)
driver.quit()
tjs = sorted(set(tjs))

In [60]:
rss = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_rumah_sakit_di_DKI_Jakarta")
rs_s = driver.find_elements(By.CSS_SELECTOR, 'tr>td:nth-child(3)>a.new[href]')
for rs in rs_s:
    rss.append(rs.text)
driver.quit()

In [63]:
list_to_scrape = malls + krls + tjs + rss
len(list_to_scrape)

519

In [73]:
url_s = ['']*len(list_to_scrape)
stars = ['']*len(list_to_scrape)
count_reviews = ['']*len(list_to_scrape)
driver = webdriver.Chrome()
for i in range(len(list_to_scrape)):
    driver.get("https://www.google.com/maps/@-6.208428,106.7824432,15z")
    time.sleep(1.81 + np.random.normal(loc=0.0,scale=0.327))
    Place = driver.find_element(By.CLASS_NAME, "tactile-searchbox-input")
    Place.send_keys(list_to_scrape[i])
    time.sleep(2.08 + np.random.normal(loc=0.0,scale=0.327))
    Place.send_keys(Keys.DOWN)
    time.sleep(0.15)
    Submit = driver.find_element(By.ID, "searchbox-searchbutton")
    Submit.click()
    time.sleep(2 + np.random.normal(loc=0.0,scale=0.127))
    try:
        url_s[i] = driver.current_url
        stars[i] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label^="Bintang"]').accessible_name # bintang = star
        count_reviews[i] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="lasan"]').accessible_name # ulasan = review
    except:
        pass

In [74]:
# check yang reviewnya kurang dari 10
# check yang nama sama urlnya beda banget

In [75]:
df = pd.DataFrame(data = {
    'name': list_to_scrape
    , 'url': url_s
    , 'stars': stars
    , 'count_reviews': count_reviews
})

In [76]:
df.head()

,name,url,stars,count_reviews
0,Central Park Jakarta,https://www.google.com/maps/place/Central+Park...,"Bintang 4,7",96.897 ulasan
1,Ciplaz Cengkareng,https://www.google.com/maps/place/Ciplaz+Cengk...,"Bintang 4,6",2.441 ulasan
2,CityWalk Gajah Mada,https://www.google.com/maps/place/Citywalk+Gaj...,"Bintang 4,5",2.978 ulasan
3,Green Sedayu Mall,https://www.google.com/maps/place/Green+Sedayu...,"Bintang 4,6",3.745 ulasan
4,HubLife Taman Anggrek,https://www.google.com/maps/place/Hublife+Tama...,"Bintang 4,6",2.225 ulasan


In [77]:
df.to_csv('gmaps_scrape_data.csv')

In [1]:
# code 2023-04-26 until here

In [5]:
df = pd.read_csv('gmaps_scrape_data.csv')
df = df.drop('Unnamed: 0', axis=1)

In [6]:
df.head()

,name,url,stars,count_reviews
0,Central Park Jakarta,https://www.google.com/maps/place/Central+Park...,"Bintang 4,7",96.897 ulasan
1,Ciplaz Cengkareng,https://www.google.com/maps/place/Ciplaz+Cengk...,"Bintang 4,6",2.441 ulasan
2,CityWalk Gajah Mada,https://www.google.com/maps/place/Citywalk+Gaj...,"Bintang 4,5",2.978 ulasan
3,Green Sedayu Mall,https://www.google.com/maps/place/Green+Sedayu...,"Bintang 4,6",3.745 ulasan
4,HubLife Taman Anggrek,https://www.google.com/maps/place/Hublife+Tama...,"Bintang 4,6",2.225 ulasan


In [7]:
df['coordinate'] = df['url'].apply(lambda x: re.search(r'@([-\d\.,]+)z', x)[1])
df['stars'] = df['stars'].apply(lambda x: re.search(r'^Bintang\s([\d,]+)', str(x))[1] if re.search(r'^Bintang\s([\d,]+)', str(x)) else None)
df['count_reviews'] = df['count_reviews'].apply(lambda x: re.search(r'([\d\.]+)\s.lasan$', str(x))[1] if re.search(r'([\d\.]+)\s.lasan$', str(x)) else None)

In [8]:
df.head()

,name,url,stars,count_reviews,coordinate
0,Central Park Jakarta,https://www.google.com/maps/place/Central+Park...,"4,7",96.897,"-6.1773686,106.7880974,17"
1,Ciplaz Cengkareng,https://www.google.com/maps/place/Ciplaz+Cengk...,"4,6",2.441,"-6.1530098,106.7256653,17"
2,CityWalk Gajah Mada,https://www.google.com/maps/place/Citywalk+Gaj...,"4,5",2.978,"-6.1453373,106.8125703,17"
3,Green Sedayu Mall,https://www.google.com/maps/place/Green+Sedayu...,"4,6",3.745,"-6.1388522,106.7261465,17"
4,HubLife Taman Anggrek,https://www.google.com/maps/place/Hublife+Tama...,"4,6",2.225,"-6.208428,106.7824432,15"


In [9]:
df['stars'] = df['stars'].str.replace(',', '.').astype('float64')

In [10]:
df.isna().sum()

name              0
url               0
stars            31
count_reviews    31
coordinate        0
dtype: int64

In [11]:
df[ df['stars'].isna() ]

,name,url,stars,count_reviews,coordinate
5,Lindeteves Trade Centre,https://www.google.com/maps/place/Lindeteves+T...,NaN,None,"-6.1452508,106.7971913,14"
89,Stasiun Cawang,"https://www.google.com/maps/place/St.+Cawang,+...",NaN,None,"-6.242039,106.8563564,17"
91,Stasiun Cipinang,https://www.google.com/maps/search/stasiun+cip...,NaN,None,"-6.2176146,106.8643371,14"
93,Stasiun Duri,"https://www.google.com/maps/place/St.+Duri,+Du...",NaN,None,"-6.1560949,106.799051,17"
121,Stasiun Pesing,"https://www.google.com/maps/place/St.+Pesing,+...",NaN,None,"-6.1612837,106.7686988,17"
124,Stasiun Rawa Buaya,https://www.google.com/maps/place/St.+Rw.+Buay...,NaN,None,"-6.1626667,106.7211309,17"
138,Halte Busway Angkutan cepat bus,"https://www.google.com/maps/@-6.208428,106.782...",NaN,None,"-6.208428,106.7824432,15"
163,Halte Busway Cipinang,https://www.google.com/maps/search/halte+buswa...,NaN,None,"-6.2084024,106.6279276,11"
165,Halte Busway Cipulir,https://www.google.com/maps/search/halte+buswa...,NaN,None,"-6.2084263,106.751543,13"
217,Halte Busway Kebayoran Lama,https://www.google.com/maps/place/Halte+Busway...,NaN,None,"-6.2383454,106.7753745,17"


For Lindeteves Trade Center, it is generally called as LTC Mall and we are going to patch it

In [12]:
df.at[5, 'stars'] = 4.6
df.at[5, 'count_reviews'] = '3.447'

Next, for sanity check, let's parse the URL and see whether it is the same as the intended place

In [17]:
df['name_url'] = df['url'].apply(lambda x: re.search(r'/place/([\+\w]+)/', x)[1] if re.search(r'/place/([\+\w]+)/', x) else None)

In [21]:
for w, x, y, z in zip(df['name'], df['name_url'], df['stars'], df['count_reviews']):
    print(w, x, y, z)

Central Park Jakarta Central+Park 4.7 96.897
Ciplaz Cengkareng None 4.6 2.441
CityWalk Gajah Mada Citywalk+Gajah+Mada 4.5 2.978
Green Sedayu Mall Green+Sedayu+Mall 4.6 3.745
HubLife Taman Anggrek Hublife+Taman+Anggrek 4.6 2.225
Lindeteves Trade Centre None 4.6 3.447
Lippo Mall Puri Lippo+Mall+Puri 4.6 49.403
Mall Ciputra Jakarta Mal+Ciputra+Jakarta 4.5 40.360
Mall Puri Indah None 4.4 1.694
Mall Taman Anggrek Mall+Taman+Anggrek 4.6 59.510
Mall Taman Palem Mall+Taman+Palem 4.3 7.025
Neo Soho Podomoro City Neo+Soho+Central+Park 4.6 24.523
Plaza Slipi Jaya Plaza+Slipi+Jaya 4.3 11.896
Seasons City None 4.3 25.791
Citywalk Sudirman Citywalk+Sudirman 4.4 20.746
FX Sudirman FX+Sudirman 4.5 37.276
Gajah Mada Plaza None 4.3 21.201
Grand Indonesia Shopping Town None 4.7 22.258
Green Pramuka Square Green+Pramuka+Square+Mall 4.5 26.735
ITC Cempaka Mas ITC+Cempaka+Mas 4.4 26.086
ITC Mangga Dua ITC+Mangga+Dua 4.4 13.374
ITC Roxy Mas ITC+Roxy+Mas 4.5 24.238
Mangga Dua Mall Mangga+Dua+Mall 4.4 23.895
M

Following [Bayes](https://en.wikipedia.org/wiki/Bayesian_average), places with no reviews will be imputed with 1 in every rating possible. Resulting in 5 reviews with 3 scores

In [27]:
df['stars'] = df['stars'].fillna(3)
df['count_reviews'] = df['count_reviews'].fillna('5')

There are malls with too few reviews. We'll check if there are any problems with the data

In [28]:
df['count_reviews'] = df['count_reviews'].str.replace('.', '').astype(int)

In [32]:
df[:84].sort_values('count_reviews').head(10)

,name,url,stars,count_reviews,coordinate,name_url
52,Pasaraya Blok M,https://www.google.com/maps/place/Pasaraya+blo...,5.0,1,"-6.2437698,106.7975297,17",Pasaraya+blok+M+gold
65,Grand Cakung,https://www.google.com/maps/place/Grand+Cakung...,5.0,3,"-6.1856989,106.9591341,17",None
43,Kalibata City Square,https://www.google.com/maps/place/Kalibata+Cit...,3.3,4,"-6.2561373,106.8483122,17",None
72,Tamini Square,https://www.google.com/maps/place/Tamini+Squar...,5.0,4,"-6.2917347,106.878946,17",None
56,Plaza Semanggi,https://www.google.com/maps/place/Plaza+Semang...,4.1,7,"-6.2194431,106.8117083,17",Plaza+Semanggi
49,Mall Metro Cipulir,https://www.google.com/maps/place/Mall+Metro+C...,4.4,471,"-6.2384832,106.7615083,17",Mall+Metro+Cipulir
41,ITC Cipulir,https://www.google.com/maps/place/ITC+Cipulir/...,4.5,1278,"-6.2372615,106.7629077,17",ITC+Cipulir
8,Mall Puri Indah,https://www.google.com/maps/place/Mall+Puri+In...,4.4,1694,"-6.1847065,106.7504279,17",None
79,Metro Sunter Plaza,https://www.google.com/maps/place/Metro+Sunter...,4.5,1987,"-6.1369481,106.8705927,17",Metro+Sunter+Plaza
4,HubLife Taman Anggrek,https://www.google.com/maps/place/Hublife+Tama...,4.6,2225,"-6.208428,106.7824432,15",Hublife+Taman+Anggrek


In [34]:
df.at[52, 'stars'] = 3 # didn't find Pasaraya Blok M in internet
df.at[52, 'count_reviews'] = 5 # didn't find Pasaraya Blok M in internet
df.at[65, 'stars'] = 4.3 # in Google Maps it is saved as Mal Grand Cakung
df.at[65, 'count_reviews'] = 7293 # in Google Maps it is saved as Mal Grand Cakung
df.at[65, 'coordinate'] = '-6.1867509,106.9563119,17' #  in Google Maps it is saved as Mal Grand Cakung
df.at[43, 'stars'] = 4.5 # in Google Maps it is saved as Mall Kalibata City Square
df.at[43, 'count_reviews'] = 6403 # in Google Maps it is saved as Mall Kalibata City Square
df.at[65, 'stars'] = 4.4 # in Google Maps there are 3 Tamini Squares
df.at[65, 'count_reviews'] = 429 # in Google Maps there are 3 Tamini Squares
df.at[65, 'stars'] = 4.3 # in Google Maps it is saved as Mall Kalibata City Square
df.at[65, 'stars'] = 7293 # in Google Maps it is saved as Mall Kalibata City Square
df.at[56, 'stars'] = 4.3 # We'll use Zoya Plaza Semanggi instead
df.at[56, 'stars'] = 27885 # We'll use Zoya Plaza Semanggi instead
df.at[8, 'stars'] = 4.6 # in Google Maps there are Puri Indah Mall and Mall Puri Indah
df.at[8, 'stars'] = 36956 # in Google Maps there are Puri Indah Mall and Mall Puri Indah

In [37]:
df = df.drop(['url', 'name_url'], axis=1)

In [39]:
df.to_csv('gmaps_scrape_data_cleaned.csv')